In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
import matplotlib.pyplot as plt
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [37]:
browser = Browser('chrome')
url = "https://housesigma.com/listings/watched-area-and-community"
browser.visit(url)
html = browser.html
soup_obj = soup(html, 'html.parser')

In [38]:
browser.execute_script('document.querySelector(".app-btn.hs_btn_login_start").click();')

email_input = WebDriverWait(browser.driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, '.form-input.medium.clear.input input[type="email"]'))
)
email = 'qwerty.kitten.m@gmail.com'
passw = 'Bootcamp2024*'
login = (email) 
email_input.send_keys(login)

In [39]:
password_input = WebDriverWait(browser.driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, '.form-input.medium.clear.input input[type="password"]'))
)
password = (passw) 
password_input.send_keys(password)

In [40]:
button = browser.find_by_css('.app-btn.hs_btn_login_submit_email').first
button.click()

In [50]:
browser.is_element_present_by_css('.pc-listing-card', wait_time=10)
html = browser.html
soup_obj = soup(html, 'html.parser')
elements = soup_obj.find_all('span', class_='text1')
for element in elements:
    print(element.text)

College Park
River Oaks
Uptown Core
Uptown Core
Old Oakville
Old Oakville
Eastlake
Old Oakville
Bronte West
West Oak Trails


In [98]:
import json
import re

data_list = []  # Initialize a list to store data for all items

# Find all articles containing property information
articles = soup_obj.find_all('article', class_='pc-listing-card')

# Loop through each article to collect data
for article in articles:
    data = {}

    # Address
    address_element = article.find('h3', class_='address')
    data['address'] = address_element.text.strip() if address_element else None

    # Text1
    text1_element = article.find('span', class_='text1')
    data['text1'] = text1_element.text.strip() if text1_element else None

    # Find all <script> tags containing JSON data within the article
    json_scripts = article.find_all('script', class_='hs-script-home-struct', type='application/ld+json')

    # Extract relevant information from each JSON script
    for script in json_scripts:
        json_data = script.string
        if json_data:
            json_dict = json.loads(json_data)
            
            # Extract and store floorSize, latitude, and longitude if available
            if 'floorSize' in json_dict:
                data['floorSize'] = json_dict['floorSize']['value']
            if 'geo' in json_dict:
                data['latitude'] = json_dict['geo']['latitude']
                data['longitude'] = json_dict['geo']['longitude']

    # Date Preview
    date_preview_element = article.find('div', class_='date-preview')
    data['date-preview'] = date_preview_element.text.strip() if date_preview_element else None

    # Highlight
    highlight_element = article.find('span', class_='highlight')
    data['highlight'] = highlight_element.text.strip() if highlight_element else None

    # Type
    type_element = article.find('p', class_='type')
    data['type'] = type_element.text.strip() if type_element else None

    # Find all <p> tags within the article
    p_elements = article.find_all('p')

    # Search for bathroom and bedroom information within <p> tags
    for p in p_elements:
        text = p.get_text()

        # Search for bathroom information
        if re.search(r'\b(?:bathroom)\b', text):
            bathroom_info = re.findall(r'\d+', text)
            if bathroom_info:
                data['bathroom'] = int(bathroom_info[0])  # Convert to integer if not empty

        # Search for bedroom information
        if re.search(r'\b(?:bedroom)\b', text):
            bedroom_info = re.findall(r'\d+', text)
            if bedroom_info:
                data['bedroom'] = int(bedroom_info[0])  # Convert to integer if not empty

    # Append the data for this article to the data list
    data_list.append(data)

# Print the list of data for all items
for i, data in enumerate(data_list, 1):
    print(f"Item {i}: {data}")


Item 1: {'address': '1277 Redbank Cres , Oakville - College Park', 'text1': 'College Park', 'floorSize': 1299, 'latitude': -79.70085, 'longitude': 43.46068, 'date-preview': '3 hours ago', 'highlight': '$848,000', 'type': 'Condo Townhouse', 'bedroom': 3, 'bathroom': 2}
Item 2: {'address': '2158 Lillykin St , Oakville - River Oaks', 'text1': 'River Oaks', 'floorSize': 4250, 'latitude': -79.7093, 'longitude': 43.47843, 'date-preview': '3 hours ago', 'highlight': '$2,769,900', 'type': 'Detached', 'bedroom': 5, 'bathroom': 6}
Item 3: {'address': '429 - 2485 Taunton Rd , Oakville - Uptown Core', 'text1': 'Uptown Core', 'floorSize': 649, 'latitude': -79.71938, 'longitude': 43.4863, 'date-preview': '6 hours ago', 'highlight': '$635,000', 'type': 'Condo Apt', 'bedroom': 2, 'bathroom': 2}
Item 4: {'address': '2501 - 297 Oak Walk Dr , Oakville - Uptown Core', 'text1': 'Uptown Core', 'floorSize': 549, 'latitude': -79.71817, 'longitude': 43.48617, 'date-preview': '6 hours ago', 'highlight': '$575,0

In [99]:

df = pd.DataFrame(data_list)

# Display the dataframe
df

,address,text1,floorSize,latitude,longitude,date-preview,highlight,type,bedroom,bathroom
0,"1277 Redbank Cres , Oakville - College Park",College Park,1299,-79.70085,43.46068,3 hours ago,"$848,000",Condo Townhouse,3,2
1,"2158 Lillykin St , Oakville - River Oaks",River Oaks,4250,-79.70930,43.47843,3 hours ago,"$2,769,900",Detached,5,6
2,"429 - 2485 Taunton Rd , Oakville - Uptown Core",Uptown Core,649,-79.71938,43.48630,6 hours ago,"$635,000",Condo Apt,2,2
3,"2501 - 297 Oak Walk Dr , Oakville - Uptown Core",Uptown Core,549,-79.71817,43.48617,6 hours ago,"$575,000",Condo Apt,1,1
4,"118 Tavistock Sq , Oakville - Old Oakville",Old Oakville,2250,-79.68256,43.43334,6 hours ago,"$2,299,000",Detached,3,2
5,"417 - 185 Robinson St , Oakville - Old Oakville",Old Oakville,1099,-79.66801,43.44496,7 hours ago,"$773,000",Condo Apt,2,2
6,"2193 Constance Dr , Oakville - Eastlake",Eastlake,1750,-79.64955,43.48113,7 hours ago,"$1,850,000",Detached,4,3
7,"417 - 185 Robinson St , Oakville - Old Oakville",Old Oakville,1099,-79.66801,43.44496,16 hours ago,"$799,500",Condo Apt,2,2
8,"3118 Victoria St , Oakville - Bronte West",Bronte West,5000,-79.71738,43.38860,16 hours ago,"$3,299,900",Detached,6,6
9,"1408 Bayberry Pl , Oakville - West Oak Trails",West Oak Trails,2531,-79.74111,43.44172,16 hours ago,"$2,079,000",Detached,4,4
